# Notæ 3 

## Formulaciones de PLE basadas en grafos

### Introducción

Los problemas provenientes de la optimización de recursos muchas veces pueden ser modelados como problemas en grafos. Generalmente, estos problemas pertenecen a la clase NP-difícil, por lo que no se esperan encontrar algoritmos eficientes que permitan resolver todas las instancias. No obstante, contar con algoritmos que resuelvan al menos instancias pequeñas es una herramienta fundamental para complementar el análisis estructural. Mediante ellos, es posible poner a prueba conjeturas, verificar resultados obtenidos de forma analítica, delegar al software actividades mecánicas, etc. 

El diseño de algoritmos específicos para resolver un problema no es un trabajo sencillo y en general requiere un gran esfuerzo. Por otro lado, una enumeración exhaustiva de las soluciones rápidamente se puede volver computacionalmente intratable incluso en instancias pequeñas. 

En este contexto, la Programación Lineal Entera (PLE) ha tomado un amplio protagonismo en el diseño de algoritmos. Esto se debe a que su sencillo formalismo es capaz de modelar un gran número de problemas de optimización de recursos. Otro factor que ha contribuido es el surgimiento de softwares de optimización (CPLEX, Gurobi, etc.), los cuales cuentan con potentes algoritmos y mecanismos adicionales para resolver PLE de una forma prácticamente transparente para el usuario.

Por estos motivos, la PLE es una herramienta fundamental en nuestra área y suele ser una de las primeras opciones a la hora de diseñar algoritmos para problemas difíciles.

El objetivo de este notebook es aprender a escribir y resolver en Python formulaciones de PLE basadas en grafos, donde las variables y restricciones dependan de ciertas estructuras de grafo, tales como, vértices, aristas, cliques, conjuntos estables, entre otras.

&#9940; Alto &#9940; No avanzar hasta haber leído los notebooks 1 y 2.

### Caso de estudio

Seguiremos como caso de estudio el problema de coloreo de vértices. Este problema es muy conocido en el área debido a todas sus aplicaciones, por ejemplo, en problemas de scheduling, de asignación de aulas, de alocación de registros, etc. Antes necesitaremos algunas definiciones.

<b>Definición.</b> Dado un grafo $G = (V,E)$ y un conjunto de colores $\mathcal C$, un coloreo de $G$ es una función $f : V \to \mathcal C$ que asigna a cada vértice de $V$ un color de $\mathcal C$ tal que vértices adyacentes tengan asignados colores distintos, es decir, para toda arista $(u,v) \in E$, $f(u) \neq f(v)$.

<b>Observación.</b> Dado un coloreo $f$ de $G$ y un color $k \in \mathcal C$, $f^{-1}(k)$ es el conjunto de vértices coloreados con $k$.

<b>Definición.</b> Dado un coloreo $f$ de $G$, $\mathcal A_f = \{k \in \mathcal C: f^{-1}(k) \neq \emptyset\}$ es el conjunto de colores usados por $f$. 

Ahora sí, podemos formular el problema de la siguiente forma:

<b>Problema de coloreo de vértices:</b><br>
<b>Entrada:</b> Un grafo $G$.<br>
<b>Salida:</b> Un coloreo $f$ de $G$ con mínimo $|\mathcal A_f|$.

Existen muchas formulaciones en la literatura que modelan este problema. Nosotros seguiremos la de <a href="https://www.sciencedirect.com/science/article/pii/S0166218X0700100X#bib3">Isabel Méndez-Díaz y Paula Zabala (2008)</a>.

Esta formulación posee dos tipos de variables. Por un lado, están las variables binarias de asignación vértice-color, dado $v \in V$ y $k \in \mathcal C$, $x_{vk} = 1$ si y solo si $v$ se pinta con $k$. Por el otro, están las variables binarias de activación de color, dado $k \in \mathcal C$, $w_k = 1$ si el color $k$ se usa. 

La función objetivo consiste en minimizar el número de colores usados, es decir: 
<p>$$\min \sum_{k \in \mathcal C}w_k.$$</p>

La primer familia de restricciones fuerza a que cada vértice se pinte con exactamente un color:
<p>$$ \sum_{k \in \mathcal C} x_{vk} = 1, ~~~~ \forall~v \in V. $$</p>

La segunda familia de restricciones obliga a que dos vértices adyacentes tengan colores diferentes:
<p>$$ x_{vk} + x_{uk} \leq 1, ~~~~ \forall~vu \in E,~  k \in \mathcal C.$$</p>

Desafortunadamente, estas restricciones no son suficientes porque falta todavía forzar a que las variables de activación de color se activen cuando un color es usado (de lo contrario el mínimo tendría valor objetivo igual a 0). Las autoras proponen una idea fantástica para esto, modificar la segunda familia de restricciones de la siguiente forma:
<p>$$ x_{vk} + x_{uk} \leq w_k, ~~~~ \forall~vu \in E,~  k \in \mathcal C.$$</p>

Así, si uno de los extremos de la arista se pinta con $k$, entonces estas restricciones fuerzan a que $w_k = 1$. A su vez, dado que $w_k$ es binaria, es decir, $w_k \in \{0,1\}$, las restricciones siguen prohibiendo que ambos extremos se pinten con $k$.

### Modelado en Python

Código escrito por Daniel. Desgrosarlo y explicarlo paso a paso.

In [1]:
import cplex
from docplex.mp.model import Model
import networkx as nx

# Generamos las variables del modelo
modelo = Model(name='coloreo')
NN = range(G.number_of_nodes())
X = [[0 for v in NN] for j in NN]
W = [0 for j in NN]
for v in NN:
    for j in NN:
        X[v][j] = modelo.binary_var(name='x_' + str(v) + '_' + str(j))
for j in NN:
    W[j] = modelo.binary_var(name='w_' + str(j))

# Generamos la fx objetivo
modelo.minimize(modelo.sum(W[j] for j in NN))

# Generamos las restricciones de asignación de colores
for v in NN:
    modelo.add_constraint(modelo.sum(X[v][j] for j in NN) == 1)

# Generamos las restricciones de adyacencias
for u, v in G.edges:
    for j in NN:
        modelo.add_constraint(X[u][j] + X[v][j] <= W[j])

modelo.export('test.lp')

# Resolvemos e imprimimos el número cromático
sol = modelo.solve(log_output=True)
chi = sol.get_objective_value()
print(f'Grafo {filename} tiene chi = {chi}')

NameError: name 'G' is not defined